# Gastos do Município de Capelinha/MG

![Capelinha, Estado de Minas Gerais!](assets/capelinha_mg_overview.jpg "Capelinha/MG")

## Introdução

Este projeto utiliza Python para fazer requisições à [API Minha Receita](https://minhareceita.org), uma iniciativa da sociedade civil que disponibiliza, de forma gratuita e acessível, dados da Receita Federal. O objetivo é obter informações sobre os credores do Município de Capelinha/MG, que foram beneficiados com recursos públicos durante o ano de 2022. Os dados empregados nesta análise foram obtidos a partir do [Portal de Dados Abertos do Tribunal de Contas do Estado de Minas Gerais](https://dadosabertos.tce.mg.gov.br).

## Disclaimer

O storytelling empregado neste estudo foi projetada com o intuito de tornar os dados apresentados e o método de elaboração compreensível para qualquer pessoa, independentemente de sua familiaridade com o assunto. Se você já possui conhecimento técnico em Python e/ou Ciência de Dados, sinta-se à vontade para ignorar quaisquer comentários que possam parecer redundantes ou enfadonhos.

## Importando bibliotecas

In [1]:
# Pandas: Data manipulation and analysis library.
# https://pandas.pydata.org
try:
    import pandas as pd
except ModuleNotFoundError:
    print('Pandas is not installed, installing now...')
    %pip install pandas
finally:
    import pandas as pd

In [2]:
# Requests: An elegant and simple HTTP library for Python, built for human beings.
# https://requests.readthedocs.io/en/latest/
try:
    import requests
except ModuleNotFoundError:
    print('Requests is not installed, installing now...')
    %pip install requests
finally:
    import requests

In [3]:
# Brazilian Utils: Utils library for Brazilian-specific businesses.
# https://pypi.org/project/brutils/
try:
    from brutils import cnpj
except ModuleNotFoundError:
    print('Brazilian Utils is not installed, installing now...')
    %pip install brutils
finally:
    from brutils import cnpj

## Configurando as fontes de dados

In [4]:
# Global variables

# Set the seed for the random number generator.
RANDOM_SEED = 42

# Default data directory.
DATA_PATH = r'data/'

# Informações referentes aos Pagamentos realizados pelos órgãos municipais.
PAYMENT = f'{DATA_PATH}dadosabertos/2022/2022.3112307.despesa.pagamento.csv'

## Importando os dados brutos

Para uma análise exploratória do conjunto de dados consulte o notebook [analise_exploratoria.ipynb](analise_exploratoria.ipynb). 

In [5]:
payment_df = pd.read_csv(PAYMENT, sep=';', encoding='utf-8')
# Cria um Dataframe com os credores únicos.
creditors_df = payment_df[['num_doc_credor', 'nom_credor']].drop_duplicates()
# Eliminando do Dataframe os credores que são pessoas físicas.
creditors_df = creditors_df[~creditors_df['num_doc_credor'].str.len().eq(11)]
creditors_df

,num_doc_credor,nom_credor
0,14323476000178,ARANAS PAPELARIA EIRELLI EPP
2,76535764000143,OI S.A. - EM RECUPERACAO JUDICIAL
5,36169783000189,CAPELINHA EXTINTORES LTDA
7,61198164000321,PORTO SEGURO CIA DE SEGUROS GERAIS
8,40222993000160,SO GESSO CAPELINHA LTDA
...,...,...
21293,07220279000168,NUTRIR DISTRIBUIDORA DE PRODUTOS PARA TERAPIA ...
21365,37830642000128,SEQUENZIA ORTO MED EIRELI
21572,27197452000161,BIATIC ASSESSORIA EM TECNOLOGIA DA INFORMACAO ...
21607,06037894000170,NILZA MARIA FERREIRA DA SILVA


In [6]:
print(f'Número de credores: {creditors_df.shape[0]}')

Número de credores: 585


In [7]:
# Usando a biblioteca brutils para validar os CNPJs.
validate_mask = creditors_df['num_doc_credor'].apply(cnpj.validate)
validate_mask.value_counts()

num_doc_credor
True     583
False      2
Name: count, dtype: int64

No contexto dos dispêndios efetuados pela administração pública, é razoável presumir que todos os CNPJs dos credores sejam válidos. Entretanto, a análise do resultado obtido na célula anterior revela a presença de CNPJs que não atendem a esse critério. Passemos à identificação desses CNPJs inválidos.

In [8]:
# Filtrando os CNPJs inválidos.
creditors_df[~validate_mask]

,num_doc_credor,nom_credor
246,0099999999999999,TRIBUNAL REGIONAL DO TRABALHO DA 03ª REGIAO VA...
3831,-1,NAO INFORMADO


Ao longo da nossa análise exploratória dos dados, identificamos que estes dois números de CNPJ correspondem às entradas para a folha de pagamentos da Prefeitura Municipal e da Câmara de Vereadores. Assim sendo, temos a possibilidade de excluir estes CNPJs do nosso conjunto de dados.

Para uma investigação mais detalhada do conjunto de dados, confira o notebook [analise_exploratoria.ipynb](analise_exploratoria.ipynb).

In [9]:
# Removendo os CNPJ inválidos.
creditors_df = creditors_df[validate_mask]

## Consultando a API Minha Receita

Vamos utilizar a [API Minha Receita](https://minhareceita.org) para obter informações sobre os credores do Município de Capelinha/MG.

A API fornece apenas um endpoint principal: `https://minhareceita.org/<número do CNPJ>`.

In [10]:
# Consultado a data de extração dos dados pela Receita Federal.
updated_at  = requests.get('https://minhareceita.org/updated')
updated_at.json()['message']

'2023-10-19 é a data de extração dos dados pela Receita Federal.'

In [11]:
def get_cnpj_data(cnpj: str) -> dict:
    """Consulta um CNPJ na API Minha Receita e retorna os dados em um dicionário.
    
    API Minha Receita: https://minhareceita.org/

    Parâmetros:
        cnpj (str): O CNPJ a ser consultado.

    Retorna:
        dict: Um dicionário contendo os dados retornados pela API.
    """

    # Criando a URL para consulta do CNPJ.
    base_url = 'https://minhareceita.org/'

    try:
        # Faz a requisição GET para a API Minha Receita.
        response = requests.get(base_url + cnpj)
        # Verifica se a resposta da API foi bem-sucedida.
        response.raise_for_status()
    except requests.exceptions.HTTPError as error:
        # Imprime o erro se a resposta da API não foi bem-sucedida.
        print(error)

    return response.json()

In [12]:
def consulta_api_minha_receita(cnpj: str) -> tuple:
    """Consulta um CNPJ na API Minha Receita e retorna os dados em três DataFrames separados.
    O primeiro DataFrame contém a maioria dos dados retornados pela API.
    O segundo e terceiro DataFrames contêm os dados de 'qsa' e 'cnaes_secundarios', respectivamente.

    Parâmetros:
        cnpj (str): O CNPJ a ser consultado.
    
    Retorna:
        tuple: Uma tupla contendo três DataFrames.
    """
    
    # Consulta o CNPJ na API Minha Receita
    response = get_cnpj_data(cnpj)
    
    # Remove 'qsa' e 'cnaes_secundarios' do dicionário e os armazena em variáveis separadas.
    qsa = response.pop('qsa')
    cnaes_secundarios = response.pop('cnaes_secundarios')

    # Converte o restante do JSON para um DataFrame.
    request_df = pd.DataFrame([response])

    # Converte 'qsa' e 'cnaes_secundarios' para DataFrames separados.
    qsa_df = pd.DataFrame(qsa)
    cnaes_secundarios_df = pd.DataFrame(cnaes_secundarios)

    # Obtém o CNPJ do DataFrame request_df
    cnpj_id = request_df['cnpj'].values[0]
    
    # Adiciona a coluna de CNPJ nos DataFrames qsa_df e cnaes_secundarios_df.
    qsa_df.insert(0, 'cnpj', cnpj_id)
    cnaes_secundarios_df.insert(0, 'cnpj', cnpj_id)

    return request_df, qsa_df, cnaes_secundarios_df

In [13]:
# Consultando todos os CNPJ's na API Minha Receita.

# Cria três listas vazias para armazenar os DataFrames retornados pela função consulta_api_minha_receita.
cnpj_reponse_data_df, qsa_reponse_data_df, cnaes_secundarios_reponse_data_dt = [], [], []

for cnpj in creditors_df['num_doc_credor']:
    request_df, qsa_df, cnaes_secundarios_df = consulta_api_minha_receita(cnpj)
    cnpj_reponse_data_df.append(request_df)
    qsa_reponse_data_df.append(qsa_df)
    cnaes_secundarios_reponse_data_dt.append(cnaes_secundarios_df)

In [14]:
# Concatena todos os DataFrames em cada lista para criar um único DataFrame para cada tipo de dado.
cnpj_data_df = pd.concat(cnpj_reponse_data_df, ignore_index=True)
qsa_data_df = pd.concat(qsa_reponse_data_df, ignore_index=True)
cnaes_secundarios_data_dt = pd.concat(cnaes_secundarios_reponse_data_dt, ignore_index=True)

C:\Users\alexc\AppData\Local\Temp\ipykernel_24932\2968485619.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  qsa_data_df = pd.concat(qsa_reponse_data_df, ignore_index=True)


In [15]:
# Salvando os DataFrames em arquivos CSV.
cnpj_data_df.to_csv(f'{DATA_PATH}output/cnpj_credores_2022.csv', index=False)
qsa_data_df.to_csv(f'{DATA_PATH}output/qsa_credores_2022.csv', index=False)
cnaes_secundarios_data_dt.to_csv(f'{DATA_PATH}output/cnaes_secundarios_credores_2022.csv', index=False)

In [16]:
# Como vamos trabalhar com essas informações no futuro, vamos utilizar um formato de arquivo mais eficiente.
cnpj_data_df.to_pickle(f'{DATA_PATH}output/cnpj_credores_2022.pkl')
qsa_data_df.to_pickle(f'{DATA_PATH}output/qsa_credores_2022.pkl')
cnaes_secundarios_data_dt.to_pickle(f'{DATA_PATH}output/cnaes_secundarios_credores_2022.pkl')

In [17]:
# Vamos salvar a lista contendo os credores únicos em um arquivo CSV.
creditors_df.to_csv(f'{DATA_PATH}output/credores_2022.csv', index=False)